<a href="https://colab.research.google.com/github/Alebasso01/SuperStore_Analysis/blob/master/SuperStore_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q dash jupyter-dash
!pip install pandasql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26772 sha256=d52c7e2b915f70980984aafbbb7b61514093bdd27d110a43d7c503d138feac5e
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [2]:
!pip install kaggle

In [3]:
# copy kaggle.json to /root/.kaggle/ folder so that kaggle cli can access it.
!mkdir /.kaggle
!mv kaggle.json /.kaggle
!mv /.kaggle /root/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d ishaanthareja007/samplesuperstore

Dataset URL: https://www.kaggle.com/datasets/ishaanthareja007/samplesuperstore
License(s): CC0-1.0
  0% 0.00/164k [00:00<?, ?B/s]
100% 164k/164k [00:00<00:00, 56.0MB/s]


In [5]:
%%capture
!unzip samplesuperstore.zip

In [6]:
import pandas as pd
import dash
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from jupyter_dash import JupyterDash
from dash import Dash, html, dcc, Input, Output
from pandasql import sqldf

In [7]:
df = pd.read_csv("SampleSuperstore.csv")

In [8]:
df.drop_duplicates(inplace = True)
df = df.drop(["Country","Postal Code","Discount"],axis=1)

In [9]:
df["ProfitMargin%"] = (df.Profit / df.Sales ) * 100

In [10]:
df

,Ship Mode,Segment,City,State,Region,Category,Sub-Category,Sales,Quantity,Profit,ProfitMargin%
0,Second Class,Consumer,Henderson,Kentucky,South,Furniture,Bookcases,261.9600,2,41.9136,16.00
1,Second Class,Consumer,Henderson,Kentucky,South,Furniture,Chairs,731.9400,3,219.5820,30.00
2,Second Class,Corporate,Los Angeles,California,West,Office Supplies,Labels,14.6200,2,6.8714,47.00
3,Standard Class,Consumer,Fort Lauderdale,Florida,South,Furniture,Tables,957.5775,5,-383.0310,-40.00
4,Standard Class,Consumer,Fort Lauderdale,Florida,South,Office Supplies,Storage,22.3680,2,2.5164,11.25
...,...,...,...,...,...,...,...,...,...,...,...
9988,Standard Class,Corporate,Athens,Georgia,South,Technology,Phones,206.1000,5,55.6470,27.00
9989,Second Class,Consumer,Miami,Florida,South,Furniture,Furnishings,25.2480,3,4.1028,16.25
9990,Standard Class,Consumer,Costa Mesa,California,West,Furniture,Furnishings,91.9600,2,15.6332,17.00
9991,Standard Class,Consumer,Costa Mesa,California,West,Technology,Phones,258.5760,2,19.3932,7.50


In [11]:
category_analysis = pd.DataFrame(df.groupby(["Category"])[["Sales","Profit","Quantity",]].sum())
category_analysis

,Sales,Profit,Quantity
Category,,,
Furniture,741306.3133,18421.8137,8020
Office Supplies,718492.0840,122291.7128,22859
Technology,836154.0330,145454.9481,6939


In [12]:
df["City"].value_counts(ascending = False)

,count
City,
New York City,914
Los Angeles,746
Philadelphia,536
San Francisco,506
Seattle,424
...,...
Glenview,1
Missouri City,1
Rochester Hills,1


In [13]:
state_name_code = {"Alabama": "AL","Alaska": "AK","Arizona": "AZ","Arkansas": "AR","California": "CA","Colorado": "CO",
                   "Connecticut": "CT","Delaware": "DE","Florida": "FL","Georgia": "GA","Hawaii": "HI","Idaho": "ID",
                   "Illinois": "IL","Indiana": "IN","Iowa": "IA","Kansas": "KS","Kentucky": "KY","Louisiana": "LA",
                   "Maine": "ME","Maryland": "MD","Massachusetts": "MA","Michigan": "MI","Minnesota": "MN",
                   "Mississippi": "MS","Missouri": "MO","Montana": "MT","Nebraska": "NE","Nevada": "NV",
                   "New Hampshire": "NH","New Jersey": "NJ","New Mexico": "NM","New York": "NY",
                   "North Carolina": "NC","North Dakota": "ND","Ohio": "OH","Oklahoma": "OK",
                   "Oregon": "OR","Pennsylvania": "PA","Rhode Island": "RI","South Carolina": "SC",
                   "South Dakota": "SD","Tennessee": "TN","Texas": "TX","Utah": "UT","Vermont": "VT",
                   "Virginia": "VA","District of Columbia": "WA","Washington": "WA","West Virginia": "WV",
                   "Wisconsin": "WI","Wyoming": "WY"}
df["state_name_code"] = df.State.apply(lambda x: state_name_code[x])

df.head(1000)

,Ship Mode,Segment,City,State,Region,Category,Sub-Category,Sales,Quantity,Profit,ProfitMargin%,state_name_code
0,Second Class,Consumer,Henderson,Kentucky,South,Furniture,Bookcases,261.9600,2,41.9136,16.00,KY
1,Second Class,Consumer,Henderson,Kentucky,South,Furniture,Chairs,731.9400,3,219.5820,30.00,KY
2,Second Class,Corporate,Los Angeles,California,West,Office Supplies,Labels,14.6200,2,6.8714,47.00,CA
3,Standard Class,Consumer,Fort Lauderdale,Florida,South,Furniture,Tables,957.5775,5,-383.0310,-40.00,FL
4,Standard Class,Consumer,Fort Lauderdale,Florida,South,Office Supplies,Storage,22.3680,2,2.5164,11.25,FL
...,...,...,...,...,...,...,...,...,...,...,...,...
996,Standard Class,Consumer,Henderson,Kentucky,South,Office Supplies,Envelopes,10.6700,1,4.9082,46.00,KY
997,Standard Class,Consumer,Henderson,Kentucky,South,Office Supplies,Storage,36.6300,3,9.8901,27.00,KY
998,Standard Class,Consumer,Henderson,Kentucky,South,Furniture,Furnishings,24.1000,5,9.1580,38.00,KY
999,Standard Class,Consumer,Henderson,Kentucky,South,Furniture,Furnishings,33.1100,7,12.9129,39.00,KY


In [14]:
df.columns = df.columns.str.replace('Sub-Category', 'SubCategory')
df.columns = df.columns.str.replace('Ship Mode', 'ShipMode')

In [15]:
query1 = "SELECT  City,Profit,Sales,Quantity,SUM(Sales),SUM(Profit),Category FROM df GROUP BY City ORDER BY SUM(Sales) DESC"
df1 = sqldf(query1)
query2= "SELECT * FROM df1 LIMIT 0, 10"
df1 = sqldf(query2)
df1

,City,Profit,Sales,Quantity,SUM(Sales),SUM(Profit),Category
0,New York City,6.2566,15.260,7,256319.0410,62013.8973,Office Supplies
1,Los Angeles,6.8714,14.620,2,175831.9010,30431.4267,Office Supplies
2,Seattle,132.5922,407.976,3,119460.2820,29121.6825,Office Supplies
3,San Francisco,2.4824,8.560,2,112577.1720,17466.1186,Office Supplies
4,Philadelphia,-1.0196,71.372,2,109061.4610,-13843.2106,Furniture
5,Houston,9.9468,29.472,3,64441.2564,-10175.1755,Office Supplies
6,Chicago,-15.2225,213.115,5,48535.9770,-6648.3318,Furniture
7,San Diego,3.5767,7.610,1,47521.0290,6377.1960,Office Supplies
8,Jacksonville,-3.6892,4.812,2,44713.1830,-2323.8350,Office Supplies
9,Springfield,35.6636,75.880,2,43054.3420,6200.6974,Office Supplies


In [16]:
df2 = df1.drop(["City","Quantity","Category","SUM(Sales)","SUM(Profit)"],axis = 1)
df2

,Profit,Sales
0,6.2566,15.260
1,6.8714,14.620
2,132.5922,407.976
3,2.4824,8.560
4,-1.0196,71.372
5,9.9468,29.472
6,-15.2225,213.115
7,3.5767,7.610
8,-3.6892,4.812
9,35.6636,75.880


In [17]:
query3 = "SELECT Category,SubCategory,City,Profit,Sales,SUM(Sales),SUM(Profit) FROM df GROUP BY Category,City ORDER BY SUM(Sales) DESC"
df3 = sqldf(query3)
df3

,Category,SubCategory,City,Profit,Sales,SUM(Sales),SUM(Profit)
0,Technology,Phones,New York City,298.6855,1029.950,109316.106,36105.5058
1,Furniture,Furnishings,New York City,40.5426,96.530,75691.049,5306.6915
2,Technology,Phones,Los Angeles,90.7152,907.152,73029.786,13908.5379
3,Office Supplies,Fasteners,New York City,6.2566,15.260,71311.886,20601.7000
4,Furniture,Furnishings,Los Angeles,14.1694,48.860,54000.041,3071.5913
...,...,...,...,...,...,...,...
1193,Office Supplies,Art,Jupiter,0.1548,2.064,2.064,0.1548
1194,Technology,Accessories,Sheboygan,0.8910,1.980,1.980,0.8910
1195,Office Supplies,Binders,Elyria,-1.3984,1.824,1.824,-1.3984
1196,Technology,Accessories,Memphis,0.4752,1.584,1.584,0.4752


In [18]:
data = pd.DataFrame({
   'lon':[40, 2, 145, 30.32, -4.03, -73.57, 36.82, -38.5],
   'lat':[43, 49, -38, 59.93, 5.33, 45.52, -1.29, -12.97],
   'name':['Buenos Aires', 'Paris', 'melbourne', 'St Petersbourg', 'Abidjan', 'Montreal', 'Nairobi', 'Salvador'],
   'value':[10, 12, 40, 70, 23, 43, 100, 43]
}, dtype=str)

data

,lon,lat,name,value
0,40,43,Buenos Aires,10
1,2,49,Paris,12
2,145,-38,melbourne,40
3,30.32,59.93,St Petersbourg,70
4,-4.03,5.33,Abidjan,23
5,-73.57,45.52,Montreal,43
6,36.82,-1.29,Nairobi,100
7,-38.5,-12.97,Salvador,43


In [19]:
df.drop_duplicates(inplace = True)
df4 = df.drop(["Segment", "City", "State", "Region", "Category", "SubCategory", "ProfitMargin%", "state_name_code", "ShipMode"],axis=1)

In [20]:
df5 = df.loc[(df.City == "Los Angeles")|(df.City == "New York City")|(df.City == "Seattle")|(df.City == "San Francisco")|(df.City == "Philadelphia")|(df.City == "Houston")|(df.City == "Chicago")|(df.City == "San Diego")|(df.City == "Jacksonville")|(df.City == "Detroit")]
query4 = "SELECT City,Region,Category,SubCategory,SUM(Sales),SUM(Profit),SUM(Quantity) FROM df5 GROUP BY City,Category,SubCategory ORDER BY SUM(Profit) DESC"
df6 = sqldf(query4)

In [21]:
df6

,City,Region,Category,SubCategory,SUM(Sales),SUM(Profit),SUM(Quantity)
0,New York City,East,Technology,Machines,37531.210,14568.8253,54
1,New York City,East,Technology,Phones,37959.130,10765.4969,319
2,New York City,East,Office Supplies,Binders,24534.128,8361.1981,560
3,Seattle,West,Technology,Copiers,17449.880,8290.4449,12
4,New York City,East,Furniture,Chairs,40149.693,6187.0539,265
...,...,...,...,...,...,...,...
162,Philadelphia,East,Office Supplies,Binders,4552.161,-3385.2340,284
163,New York City,East,Furniture,Tables,10964.094,-3542.7216,78
164,Philadelphia,East,Technology,Phones,19432.182,-3555.0398,226
165,Chicago,Central,Office Supplies,Binders,3164.766,-4905.1274,179


In [22]:
state_data = df[['Profit', 'state_name_code','Sales']].groupby(['state_name_code']).sum()

app = dash.Dash(__name__)

app.layout = html.Div(
    children=[
        # Header Section
        html.Div(
            children=[html.H1("SUPERSTORES IN USA")],
            style={
                'text-align': 'center',
                'color': 'white',
                'font-family': 'Arial',
                'background-color': '#5f819e'
            }
        ),

        # First Two Graphs Section
        html.Div(
            children=[
                # Box 1 (Category Dropdown and Graph)
                html.Div(
                    children=[
                        html.Div(
                            children=[
                                html.Div(
                                    children=[
                                        html.P('Choose Category:'),
                                    ],
                                    style={'font-size': 20, 'font-family': 'Arial', 'left': 0, 'width': 200, 'color': "#284B63"}
                                ),
                                html.Div(
                                    children=[
                                        dcc.Dropdown(
                                            options=[{'label': col, 'value': col} for col in df2.columns],
                                            value='Sales',
                                            id='input_value'
                                        )
                                    ],
                                    style={'font-size': 20, 'color': "#284B63", 'position': "relative", 'top': 15, 'left': 0, 'width': 350}
                                ),
                            ],
                            style={'display': "flex"}
                        ),
                        dcc.Graph(id='graph1'),
                    ],
                    style={'width': 550}
                ),

                # Box 2 (Category Dropdown and Graph)
                html.Div(
                    children=[
                        html.Div(
                            children=[
                                html.Div(
                                    children=[
                                        html.P('Choose Category:'),
                                    ],
                                    style={'font-size': 20, 'font-family': 'Arial', 'left': 0, 'width': 300, 'color': "#284B63"}
                                ),
                                html.Div(
                                    children=[
                                        dcc.Dropdown(
                                            options=[{'label': col, 'value': col} for col in df4.columns],
                                            value='Profit',
                                            id='input'
                                        ),
                                    ],
                                    style={'font-size': 20, 'color': "#284B63", 'position': "relative", 'top': 15, 'left': 0, 'width': 250}
                                ),
                            ],
                            style={'display': "flex"}
                        ),
                        dcc.Graph(id='graph2'),
                    ],
                    style={'width': 550}
                ),
            ],
            style={'display': "inline-block"}
        ),

        # Third and Fourth Graphs Section
        html.Div(
            children=[
                # Box 3 (Region Dropdown and Graph)
                html.Div(
                    children=[
                        html.Div(
                            children=[
                                html.Div(
                                    children=[
                                        html.P('Choose Region:'),
                                    ],
                                    style={'font-size': 20, 'font-family': 'Arial', 'left': 0, 'width': 200, 'color': "#284B63"}
                                ),
                                html.Div(
                                    children=[
                                        dcc.Dropdown(
                                            options=[{'label': region, 'value': region} for region in df.Region.unique()],
                                            value='East',
                                            id='input_region'
                                        ),
                                    ],
                                    style={'font-size': 20, 'color': "#284B63", 'position': "relative", 'top': 15, 'left': 0, 'width': 400}
                                ),
                            ],
                            style={'display': "flex"}
                        ),
                        dcc.Graph(id='graph3'),
                    ],
                    style={'position': "absolute", 'width': 550, 'top': 78, 'left': 592}
                ),

                # Box 4 (City Dropdown and Graph)
                html.Div(
                    children=[
                        html.Div(
                            children=[
                                html.Div(
                                    children=[
                                        html.P('Choose City:'),
                                    ],
                                    style={'font-size': 20, 'font-family': 'Arial', 'left': 0, 'width': 150, 'color': "#284B63"}
                                ),
                                html.Div(
                                    children=[
                                        dcc.Dropdown(
                                            options=[{'label': city, 'value': city} for city in df6.City.unique()],
                                            value='New York City',
                                            id='input_City'
                                        ),
                                    ],
                                    style={'font-size': 20, 'color': "#284B63", 'position': "relative", 'top': 15, 'left': 0, 'width': 400}
                                ),
                            ],
                            style={'display': "flex"}
                        ),
                        dcc.Graph(id='graph4'),
                    ],
                    style={'width': 550, 'position': "absolute", 'top': 347, 'left': 592}
                ),
            ],
            style={'display': "inline-block"}
        )
    ],
    style={
        'height': 615,
        'width': 1143,
        'position': 'absolute',
        'top': 0,
        'left': 0,
        'right': 0,
        'background-color': "#edf6f9"
    }
)

#------------- CALLBACKS -----------------------------------------------------------

# GRAFH 1 --------------------------------------------------------
@app.callback(
    Output('graph1', 'figure'),
    Input('input_value', 'value')
)
def update_figure_output(input_value):
    fig1 = go.Figure(data=go.Choropleth(
        locations=state_data.index,
        z=state_data[input_value],
        locationmode='USA-states',
        colorscale='blues'
    ))

    fig1.update_layout(
        geo_scope='usa',
        height=205,
        margin=dict(l=0, r=0, t=0, b=0),
        paper_bgcolor="#edf6f9"
    )
    return fig1


# GRAFH 2 ------------------------------------------------
@app.callback(
    Output('graph2', 'figure'),
    Input('input', 'value')
)
def update_input_text(input_value):
    fig2 = px.histogram(
        data_frame=df,
        x='Category',
        y=input_value,
        width=550,
        color="Category",
        color_discrete_map={
            'Technology': '#ae2012',
            'Office Supplies': '#f4a261',
            'Furniture': '#fee440'
        }
    )

    fig2.update_layout(
        height=205,
        margin=dict(l=0, r=0, t=0, b=0),
        paper_bgcolor="#edf6f9"
    )
    return fig2


# GRAFH 3-------------------------------------------
@app.callback(
    Output('graph3', 'figure'),
    Input('input_region', 'value')
)
def update_input_text(input_region):
    fig3 = px.pie(
        data_frame=df[df.Region == input_region],
        values="Quantity",
        names="ShipMode",
        color="ShipMode",
        color_discrete_map={
            'First Class': '#968e85',
            'Second Class': '#8cc084',
            'Standard Class': '#008000',
            'Same Day': '#bb9457'
        },
        width=550,
        hole=0.5
    )

    fig3.update_layout(
        height=205,
        paper_bgcolor="#edf6f9",
        margin=dict(l=0, r=0, t=0, b=0),
        annotations=[dict(text=input_region, x=0.5, y=0.5, font_size=20, showarrow=False)]
    )
    return fig3


# GRAFH 4-------------------------------------
@app.callback(
    Output('graph4', 'figure'),
    Input('input_City', 'value')
)
def update_input_text(input_City):
    fig4 = px.bar(
        data_frame=df6[df6.City == input_City],
        x="SubCategory",
        y="SUM(Profit)",
        color="Category",
        width=550,
        color_discrete_map={
            'Technology': '#ae2012',
            'Office Supplies': '#f4a261',
            'Furniture': '#fee440'
        }
    )

    fig4.update_layout(
        height=205,
        margin=dict(l=0, r=0, t=0, b=0),
        paper_bgcolor="#edf6f9"
    )
    return fig4


# Esegui l'app
if __name__ == "__main__":
    app.run_server(debug=True)

<IPython.core.display.Javascript object>